# MCMC para Ising 

En el notebook anterior, vimos cómo muestrear de una distribución de probabilidad $\pi$ dada, usando el algoritmo de Metropolis, que es un ejemplo de un método de "Markov Chain Monte Carlo" ("Monte Carlo usando cadenas de Markov",  MCMC).
Apliquemos esto al modelo de Ising.

$\newcommand{\ss}{\pmb \sigma}
\newcommand{\tt}{\pmb \tau}$

[1] Escribe la probabilidad de aceptación, $\alpha(\ss \to \tt)$, para la distribución de Boltzmann, en términos de la diferencia de energías, $\Delta E := E(\tt) - E(\ss)$.

[2] En el caso de voltear un solo espín, encuentra una expresión analítica para $\Delta E$, en términos del valor actual del espín que se va a voltear (antes de que se cambie su valor) y los valores de sus espines. 

[3] Así, implementa el algoritmo de Metropolis para simular el modelo de Ising a una temperatura $T$ dada. 

Nota que el resultado de la pregunta [2] muestra que, una vez propuesto un espín por voltear, *no* es necesario voltear el espín para calcular la $\Delta E$, y, por lo tanto, se requiere voltearlo *sólo* si el cambio propuesto resulte ser aceptado. Se puede voltear al modificar la misma configuración.

[4] Durante la simulación, rastrea los valores de la magnetización $M(t)$ y la energía $E(t)$. Dibuja estas cantidades como función del tiempo.

[5] Quisiéramos calcular estimados de valores promedio como $\langle E \rangle_\beta$. Viendo tus gráficas, propón cómo hacer esto e impleméntalo. 

[6] Así, calcula $\langle E \rangle_\beta$ para distintos valores de $\beta$ entre $0$ y $5$, poniendo $J = 1$.

[7] Haz lo mismo con la magnetización. ¿Por qué no da un resultado interesante? ¿Qué podrías hacer al respecto? Impleméntalo y dibuja las gráficas.

[8] Cambia gradualmente la temperatura inversa de $5$ a $0$ y haz una animación de algunas **configuraciones** representativas para distintas temperaturas.

[9] Interpreta todo esto físicamente. ¿Qué estamos viendo?

## -Respuestas.

### 1.-

Sabemos que si $\pi(\ss)$ es la probabilidad estacionaria de estar en el lugar $\ss$, entonces la probabilidad de aceptaci\'on de pasar a $\tt$ es
\begin{equation}
\alpha(\ss\rightarrow\tt)=\frac{\pi(\tt)}{\pi(\ss)}
\end{equation}
pero $\pi(\ss)=e^{-\beta E(\ss)}$por lo tanto
\begin{equation}
\alpha(\ss\rightarrow\tt)=e^{-\beta \Delta E}
\end{equation}


### 2.- 

La energía del estado $\ss$ está dada por:
\begin{equation}
E(\ss)=-\sum_{\langle i,j\rangle} J_{i,j}\sigma_i \sigma_j
\end{equation}
con $i,j$ vecinos en la red de espines. Para simplificar el  calculo asumimos que $J_{i,j}=1\ \ \forall i,j$.
Si volteamos el espín i' entonces, cuatro de los elementos de la suma cambian de signo (los relacionados con los vecinos de $i'$), esto es
\begin{equation}
\Delta E=E(\tau)-E(\sigma)=-\sum_{\text{$j$ vc a i'}} \sigma_j(\tau_{i'}-\sigma_{i'})
\end{equation}
(vc se refiere a vecinos cercanos), pero $\tau_{i'}-\sigma_{i'}=-2\sigma_{i'}$, por lo tanto
\begin{equation}
\Delta E=E(\tau)-E(\sigma)=2\sigma_{i'}\sum_{\text{$j$ vc a i'}} \sigma_j
\end{equation}

### 3.-

In [1]:
using PyPlot

INFO: Loading help data...


In [2]:
#Temperatura y beta
#kB=1.3806488*10.0^(-23)
kB=1
β(T)=1/(kB*T);
T=0.2
β(T)

5.0

In [3]:
####Configuracion y funciones de flip,energia y magnetización####

#Configuracion aleatoria en un matriz
function config(N,M)
    2*int(rand(N,M))-1
end

#Hacemos nuestro propio modulo para tener fronteras periodicas
function modulo(i::Int64,L::Int64)
    i = i>L ? 1 : i<1 ? L : i
end

#Flip a la configuracion
function flip!(σ::Array{Int64,2},i::Int64=0,j::Int64=0)               
  N=size(σ)[1]
  M=size(σ)[2]
  if i==0 && j==0
    i,j=int((N-1)*rand())+1,int((M-1)*rand())+1
  end
  σ[i,j]=-σ[i,j]
  σ
end

#Calculo de Energia
function energia(σ)
    L1=size(σ)[1]
    L2=size(σ)[2]
    E=0
    for i in 1:L1
        for j in 1:L2
            E+=-σ[i,j]*(σ[modulo(i+1,L1),j]+σ[modulo(i-1,L1),j]+
                σ[i,modulo(j+1,L2)]+σ[i,modulo(j-1,L2)])
        end
    end
    E
end

function magnet(σ)
    sum(σ)
end

#Cambio de energia en un flip propuesto
function δE(σ::Array{Int64,2},i::Int64=0,j::Int64=0)
    if i==0 && j==0
    i,j=int((size(σ,1)-1)*rand())+1,int((size(σ,2)-1)*rand())+1
    end
    L1=size(σ)[1]
    L2=size(σ)[2]
    
    2*σ[i,j]*(σ[modulo(i+1,L1),j]+σ[modulo(i-1,L1),j]+
    σ[i,modulo(j+1,L2)]+σ[i,modulo(j-1,L2)]),
    i,j
end

δE (generic function with 3 methods)

In [4]:
#Proceso estocastico
function metropolis!(σ::Array{Int64,2},T::Float64)    
    ΔE,i,j=δE(σ)
    ΔM=0
    α=exp(-β(T)*ΔE)
    r=rand()
    cambio=false
    if r<α
        σ[i,j]=-σ[i,j]
        cambio=true
        ΔM=σ[i,j]<0?-2:2
    else
        σ[i,j]=σ[i,j]
        cambio=false
    end
    return cambio,ΔE,ΔM
end

metropolis! (generic function with 1 method)

### 4.-

In [5]:
######Evolucion de una cadena#######

function iteracion_con_E_y_M(espines::Array{Int64,2},iteraciones::Int64,T::Float64=30.0)
    Es::Array{Int64}=Array(Int64, iteraciones)
    Ms::Array{Int64}=Array(Int64, iteraciones)
    cambios::Array{Bool}=Array(Bool, iteraciones)
    E=energia(espines)
    M=magnet(espines)
    
    for i in 1:iteraciones
        cambio,ΔE,ΔM=metropolis!(espines,T)
        cambios[i]=cambio
        if cambio
            E+=ΔE
            M+=ΔM
        end
        Es[i]=E
        Ms[i]=M
    end
    
    return Es,Ms,cambios
end

iteracion_con_E_y_M (generic function with 2 methods)

In [44]:
#Valores iniciales
ancho=50
largo=50
N=ancho*largo
veces=int(1e6)
espines=config(ancho,largo)

Ts=[10.0,1.0,0.5,1/3,1/4,1/5]
Es=Array(Array{Float64,1},length(Ts))
Ms=Array(Array{Float64,1},length(Ts))
ds=Array(Array{Bool,1},length(Ts))

#Simulacion
for i in 1:length(Ts)
    Es[i],Ms[i],ds[i]=iteracion_con_E_y_M(espines,veces,Ts[i]);
end

In [47]:
#Ploteo
fig = figure("primerasTs",figsize=(12,5))
subplot(231)
plot(Es[1])
title("T="Ts[1])
ylabel("Energía")

subplot(232)
plot(Es[2])
title(Ts[2])

subplot(233)
plot(Es[3])
title(Ts[3])

subplot(234)
plot(Ms[1])
ylabel("Magnetización")
xlabel("Iteraciones")

subplot(235)
plot(Ms[2])
xlabel("Iteraciones")

subplot(236)
plot(Ms[3])
xlabel("Iteraciones")

fig = figure("segundasTs",figsize=(12,5))
subplot(231)
plot(Es[4])
title(Ts[4])
ylabel("Energía")

subplot(232)
plot(Es[5])
title(Ts[5])

subplot(233)
plot(Es[6])
title(Ts[6])

subplot(234)
plot(Ms[4])
ylabel("Magnetización")
xlabel("Iteraciones")

subplot(235)
plot(Ms[5])
xlabel("Iteraciones")

subplot(236)
plot(Ms[6])
xlabel("Iteraciones")

LoadError: PyError (:PyObject_Call) <class 'AttributeError'>
AttributeError("'float' object has no attribute 'items'",)
  File "/home/ernesto/anaconda3/lib/python3.4/site-packages/matplotlib/pyplot.py", line 1349, in title
    l =  gca().set_title(s, *args, **kwargs)
  File "/home/ernesto/anaconda3/lib/python3.4/site-packages/matplotlib/axes/_axes.py", line 145, in set_title
    title.update(fontdict)
  File "/home/ernesto/anaconda3/lib/python3.4/site-packages/matplotlib/artist.py", line 754, in update
    for k, v in six.iteritems(props):
  File "/home/ernesto/anaconda3/lib/python3.4/site-packages/six.py", line 558, in iteritems
    return iter(d.items(**kw))

while loading In[47], in expression starting on line 5


### 5.-